# **Soft Labels**

### ¿Qué son los Soft Labels?
Los **soft labels** (etiquetas suaves) son probabilidades asociadas a cada clase frente a etiquetas duras (hard labels). 
- En lugar de asignar una clase específica (por ejemplo, `0` o `1`), las soft labels representan la probabilidad de pertenencia de una muestra a cada clase.

Por ejemplo:
- Hard label: `[0]` (la muestra pertenece a la clase 0).
- Soft label: `[0.8, 0.2]` (la muestra tiene un 80% de probabilidad de pertenecer a la clase 0 y un 20% de pertenecer a la clase 1).

### ¿Por qué usar Soft Labels?
1. **Incorporación de incertidumbre**: Las soft labels permiten modelar la incertidumbre en las etiquetas.
2. **Mejora en el entrenamiento**: Algunos modelos pueden beneficiarse de soft labels para aprender patrones más robustos.
3. **Transferencia de conocimiento**: En aprendizaje por transferencia, las soft labels generadas por un modelo maestro pueden usarse para entrenar un modelo estudiante (técnica conocida como *knowledge distillation*).

## 2. Ejemplo Práctico con Scikit-Learn

En este ejemplo, generaremos datos sintéticos y usaremos soft labels para entrenar un modelo de regresión logística.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
df = pd.read_csv('../../data/titanic/train.csv')
df.columns

df.dropna(inplace=True)

In [4]:
X = df.drop(['PassengerId', 'Survived', 'Pclass', 'Name',
            'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)

y = df['Survived']
X, y

(        Sex   Age  SibSp
 1    female  38.0      1
 3    female  35.0      1
 6      male  54.0      0
 10   female   4.0      1
 11   female  58.0      0
 ..      ...   ...    ...
 871  female  47.0      1
 872    male  33.0      0
 879  female  56.0      0
 887  female  19.0      0
 889    male  26.0      0
 
 [183 rows x 3 columns],
 1      1
 3      1
 6      0
 10     1
 11     1
       ..
 871    1
 872    0
 879    1
 887    1
 889    1
 Name: Survived, Length: 183, dtype: int64)

In [5]:
lenc = LabelEncoder()
stadscl = StandardScaler()

X['Sex'] = lenc.fit_transform(X['Sex'])
X = stadscl.fit_transform(X)

print('X:', X[:5])

X: [[-1.03901177  0.14906507  0.83362754]
 [-1.03901177 -0.0432295   0.83362754]
 [ 0.96245301  1.17463611 -0.7230443 ]
 [-1.03901177 -2.03027338  0.83362754]
 [-1.03901177  1.43102886 -0.7230443 ]]


In [6]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Clasificador clásico

In [7]:
class_model = LogisticRegression(random_state=42)
class_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
y_pred = class_model.predict(X_test)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7838


### Crear Soft Labels
Generamos soft labels simulando probabilidades para cada clase. Usaremos un modelo maestro (en este caso, también una regresión logística) para generar estas probabilidades.


In [8]:
# Entrenar un modelo maestro para generar soft labels
master_model = LogisticRegression(random_state=42)
master_model.fit(X_train, y_train)

# Generar soft labels usando predict_proba: devuelve la estimación de probabilidad
soft_labels_train = master_model.predict_proba(X_train)

# Mostrar las primeras 5 soft labels
print("Soft Labels (primeras 5 filas):")
print(soft_labels_train[:5])

Soft Labels (primeras 5 filas):
[[0.58117418 0.41882582]
 [0.49739343 0.50260657]
 [0.38214946 0.61785054]
 [0.4892546  0.5107454 ]
 [0.53349765 0.46650235]]


### Entrenar un Modelo Estudiante con Soft Labels
Usamos las soft labels generadas para entrenar un modelo lineal

In [10]:
# Entrenar un modelo con soft labels
reg_model = LinearRegression()
# Usamos la probabilidad de la clase 1
reg_model.fit(X_train, soft_labels_train[:, 1])

# Predecir en el conjunto de prueba
y_pred = reg_model.predict(X_test)
print(y_pred)
y_pred_proba = (y_pred > 0.5).astype(int)

# Evaluar el rendimiento
accuracy = accuracy_score(y_test, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")

[0.49545973 0.94875295 0.98016091 0.42547598 0.4314402  0.3553006
 0.49545973 0.90541655 0.91893186 0.40301434 0.39406802 0.88314655
 0.43283544 0.91893186 0.41951176 0.49844184 0.97857404 0.44774598
 0.47160286 0.98453826 0.89945233 0.90839866 0.44178176 0.57915042
 0.37319325 1.03085677 0.93523764 0.42687122 0.94716608 0.89209287
 0.95611241 0.5718379  0.45967442 0.95611241 0.4537102  0.87857757
 0.97996928]
Accuracy: 0.8108


## 3. Resultados y Conclusión

### Resultados
- El modelo estudiante se entrena con soft labels generadas por el modelo maestro.
- Las soft labels permiten al modelo estudiante aprender patrones más suaves y generalizables.

### Conclusión
- Los soft labels son útiles para incorporar incertidumbre en el proceso de entrenamiento.
- Técnicas como *knowledge distillation* utilizan soft labels para transferir conocimiento de un modelo grande a uno más pequeño.

## Ejercicio:
1. Entrenar un modelo de forecasting para predecir predecir si el consumo de combustible subirá o bajar a 12 meses vista usando sof labels.

- Usa el dataset 'fuel_consumption'.
- Usa frecuancia mensual.

ref. https://github.com/skforecast/skforecast-datasets

In [ ]:
import pandas as pd
from skforecast.datasets import fetch_dataset
from tabulate import tabulate

# Cargar el dataset fuel_consumption
data = fetch_dataset("fuel_consumption")

# Crear la variable objetivo: 1 si el consumo sube, 0 si baja
data['target'] = (data['GLPs'].diff() > 0).astype(int)

# Eliminar valores faltantes generados por la diferenciación
data = data.dropna()

# Mostrar los primeros registros
print(tabulate(data.head(10),   headers='keys', tablefmt='psql'))